In [ ]:
# TODO add the OPEN_AI_KEY from moodle
OPEN_AI_KEY = ''

In [ ]:
import cv2
import base64
from openai import OpenAI

# OpenAI API client
client = OpenAI(api_key=OPEN_AI_KEY)

# Function to encode image in Base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Extract frames from video
def extract_frames(video_path, num_frames=8):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    interval = max(1, total_frames // num_frames)
    
    frames = []
    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
        ret, frame = cap.read()
        if ret:
            frame_path = f"frame_{i}.jpg"
            cv2.imwrite(frame_path, frame)
            frames.append(encode_image(frame_path))
    
    cap.release()
    return frames

# Video file and results below
#video_path = "1_ThrowIn_25.mp4"
#{
#  "Soccer Corner": 0.05,
#  "Soccer Free Kick": 0.10,
#  "Soccer Throw In": 0.85
#}

#video_path = "1_FreeKick_1.mp4"
#{
#  "Soccer Corner": 0.05,
#  "Soccer Free Kick": 0.90,
#  "Soccer Throw In": 0.05
#}

video_path = "1_CornerKick_3.mp4"
#{
#  "Soccer Corner": 0.05,
#  "Soccer Free Kick": 0.85,
#  "Soccer Throw In": 0.10
#}

frames_base64 = extract_frames(video_path, num_frames=8)

In [ ]:
# Prompt for classification
prompt = "Classify the soccer action in the images. The images are frames from a video. Options: 'Soccer Corner', 'Soccer Free Kick', 'Soccer Throw In'. Return a probability as JSON."

# Prepare request payload
message_content = [{"type": "text", "text": prompt}]
for frame_b64 in frames_base64:
    message_content.append({
        "type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{frame_b64}"}
    })

# Send request to OpenAI
completion = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",  # Ensure you use a model that supports vision
    messages=[{"role": "user", "content": message_content}]
)

# Print result
print(completion.choices[0].message.content)
